<div style ="font-family:Trebuchet MS; background-color : #f8f0fa; border-left: 5px solid #1b4332; padding: 12px; border-radius: 50px 50px;">
    <h2 style="color: #1b4332; font-size: 48px; text-align: center;"><b> Dog Bread Classification Using Transfer Learning
 </b></h2>

# Images Exploration & Analysis

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
import matplotlib.image as mpimg
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Activation, BatchNormalization,Dense, Dropout,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
labels = pd.read_csv("/kaggle/input/dog-breed-identification/labels.csv")
labels.head()

In [ ]:
labels.shape

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')

In [ ]:
sample_submission.head()

In [ ]:
breed_counts = labels['breed'].value_counts()


top_10 = breed_counts.head(10)


bottom_10 = breed_counts.tail(10)

fig, axes = plt.subplots(1, 2, figsize=(20, 8))


sns.barplot(y=top_10.index, x=top_10.values, orient='h', ax=axes[0])
axes[0].set_title('Distribution of Top 10 Breeds')
axes[0].set_xlabel('Number of Images')
axes[0].set_ylabel('Breed')

sns.barplot(y=bottom_10.index, x=bottom_10.values, orient='h', ax=axes[1])
axes[1].set_title('Distribution of Least 10 Breeds')
axes[1].set_xlabel('Number of Images')
axes[1].set_ylabel('Breed')

plt.tight_layout()
plt.show()


In [ ]:
image_dir_train = '/kaggle/input/dog-breed-identification/train'
image_dir_test  = '/kaggle/input/dog-breed-identification/test'
image_files_train = [f for f in os.listdir(image_dir_train) if f.endswith('.jpg')]
image_files_test = [f for f in os.listdir(image_dir_test) if f.endswith('.jpg')]

image_files_train[:5]


**labels length = images_files length**

In [ ]:
len(image_files_train)

In [ ]:
len(image_files_test)

In [ ]:
def display_random_images(dir,image_files,num_samples=5):

    sample_images = random.sample(image_files, num_samples)
    fig, axes = plt.subplots(1, num_samples, figsize=(num_samples * 3, 5))

    for i, image_file in enumerate(sample_images):
        img_path = os.path.join(dir, image_file)
        img = mpimg.imread(img_path)

        axes[i].imshow(img)
        axes[i].axis('off')
        axes[i].set_title(image_file.split('.')[0], fontsize=10)

    plt.tight_layout(w_pad=2)
    plt.show()

display_random_images(image_dir_train,image_files_train,num_samples=5)


**display some of the test images**

In [ ]:
display_random_images(image_dir_test,image_files_test,num_samples=5)

**Label Encoder on Labels**

In [ ]:
label_encoder = LabelEncoder()
labels['breed_label'] = label_encoder.fit_transform(labels['breed'])


In [ ]:
max_value = labels['breed_label'].max()
min_value = labels['breed_label'].min()

print(f"Maximum value: {max_value}")
print(f"Minimum value: {min_value}")


In [ ]:
labels.head()

In [ ]:
labels['breed_label'][0]

In [ ]:
labels['breed'][0]

# Data PreProcessing

create a dictionary to map breeds to labels

In [ ]:
class_names_label = dict(zip(label_encoder.transform(label_encoder.classes_),label_encoder.classes_))


In [ ]:
for k,v in class_names_label.items():
  print( str(k) + ': ' + v)
  if k == 5 :
    break




In [ ]:
IMAGE_SIZE = (224, 224)

def load_data(image_dir_train,  labels_df):
    """
    Load and preprocess the data:
    - Train images and their labels.
    - Test images for evaluation.
    """

    def process_images(image_dir):
        images = []
        print("Loading images from {}".format(image_dir))
        for image_file in tqdm(os.listdir(image_dir)):
            img_path = os.path.join(image_dir, image_file)

            # Open and resize the image
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, IMAGE_SIZE)

            images.append(image)

        return np.array(images, dtype='float32')

    # Process train images and labels
    train_images = process_images(image_dir_train)
    labels = []
    for image_file in tqdm(os.listdir(image_dir_train)):
        image_id = image_file.split('.')[0]
        label = labels_df.loc[labels_df['id'] == image_id, 'breed_label'].values[0]
        labels.append(label)
    train_labels = np.array(labels, dtype='int32')
    return train_images, train_labels



train_images, train_labels= load_data(image_dir_train,labels)
print(f"Train images shape: {train_images.shape}")
print(f"Train labels shape: {train_labels.shape}")
# print(f"Test labels shape: {test_images.shape}")

In [ ]:
train_images[0]


In [ ]:
max(train_labels),min(train_labels),len(train_labels)

In [ ]:
target_breed = "shih-tzu" 
target_label = labels[labels['breed'] == 'shih-tzu'].iloc[0, 2] 
count = 0  
max_images = 10 
num_columns = 5  # Number of columns in the grid
num_rows = max_images // num_columns + int(max_images % num_columns > 0)  # Calculate the number of rows needed

# Create a grid of subplots
fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 6))

for index in range(len(train_images)):
    if train_labels[index] == target_label:
        image = train_images[index]
        row = count // num_columns
        col = count % num_columns
        ax = axes[row, col]

        ax.imshow(image.astype('uint8'))  # Convert to uint8 if needed
        ax.set_xticks([])
        ax.set_yticks([])
        ax.grid(False)
        ax.set_title('Label #{} : '.format(train_labels[index]) + class_names_label[train_labels[index]])

        count += 1
        if count >= max_images:
            break
for i in range(count, num_rows * num_columns):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.show()

In [ ]:
labels[labels['breed'] == 'shih-tzu']

**One Hot Encoder**

In [ ]:
train_labels_one_hot = to_categorical(train_labels, num_classes=120)
train_labels_one_hot[0],train_labels[0]

# Spliting Train & Validation 

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels_one_hot, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

In [ ]:
# model = Sequential([
#     Conv2D(32, (3, 3),input_shape = (224,224,3) , activation='relu'),
#     MaxPooling2D(2, 2),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Conv2D(128, (3, 3),  activation='relu'),
#     MaxPooling2D(2, 2),
#     Conv2D(256, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.4),  # Dropout to prevent overfitting
#     Dense(120, activation='softmax')
# ])

In [ ]:
# model.compile(optimizer=Adam(learning_rate=0.0001), 
#               loss='categorical_crossentropy', 
#               metrics=['accuracy'])

# model.summary()

In [ ]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    horizontal_flip=True,     # Randomly flip half the images horizontally
    rotation_range=20,        # Random rotations up to 20 degrees
    width_shift_range=0.2,    # Randomly shift images horizontally by 20%
    height_shift_range=0.2,   # Randomly shift images vertically by 20%
    shear_range=0.2,          # Apply random shear transformations
    zoom_range=[0.9, 1.1],    # Slightly conservative zoom
    brightness_range=[0.8, 1.2], # Randomly change brightness
    fill_mode='nearest'       # Fill in new pixels using reflection to avoid artifacts
)


In [ ]:
datagen2 = ImageDataGenerator(
    rescale=1.0/255.0,
    horizontal_flip=True,     # Randomly flip half the images horizontally
    zoom_range=[0.9, 1.1],    # Slightly conservative zoom
    brightness_range=[0.8, 1.2], # Randomly change brightness
    fill_mode='nearest'       # Fill in new pixels using reflection to avoid artifacts
)

In [ ]:
## without data augmentation 
datagen3 = ImageDataGenerator(
    rescale=1.0/255.0,
)

In [ ]:
X_val = X_val / 255.0


In [ ]:
train_generator = datagen.flow(X_train, y_train, batch_size=32)
train_generator2 = datagen2.flow(X_train, y_train, batch_size=32)
train_generator3 = datagen3.flow(X_train, y_train, batch_size=32)


# Show Some Augmentations

In [ ]:
def display_augmented_images(datagen, image, num_images=4, grid_size=(2, 2), target_size=(150, 150)):
    """
    Displays augmented images in a grid layout.

    Parameters:
        datagen (ImageDataGenerator): The data generator with the augmentation settings.
        image (numpy array): The image to be augmented.
        num_images (int): Number of augmented images to display.
        grid_size (tuple): The size of the grid (rows, cols).
        target_size (tuple): The target size for resizing the images (height, width).
    """
    # Ensure the image has the correct shape for the generator
    img = image.reshape((1,) + image.shape)
    
    # Create the grid layout
    fig, axes = plt.subplots(grid_size[0], grid_size[1], figsize=(8, 8))
    axes = axes.flatten()  # Flatten the grid to make indexing easier
    
    # Generate and plot the images
    for i, batch in enumerate(datagen.flow(img, batch_size=1)):
        if i >= num_images:
            break

        # Normalize the image data to the range [0, 1] if necessary
        image_to_display = batch[0] / 255.0 if batch[0].max() > 1 else batch[0]
        
        # Remove the color channels dimension if it's grayscale
        if image_to_display.shape[-1] == 1:
            image_to_display = image_to_display.squeeze()

        # Display the image
        axes[i].imshow(image_to_display)
        axes[i].axis('off')  # Hide axes
    
    plt.tight_layout()
    plt.show()


display_augmented_images(datagen, X_train[0], num_images=8, grid_size=(4, 2))

**try another combination**

In [ ]:
display_augmented_images(datagen2, X_train[0], num_images=4, grid_size=(2, 2))

In [ ]:
# Plot training & validation loss values
def plot_loss(history,model) :
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")

In [ ]:
def plot_acc(history,model) :
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")

# Transfer Learning 

**callbacks**

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch < 8:
        return lr
    else:
        return lr * float(tf.math.exp(-0.1))

lr_callback = LearningRateScheduler(scheduler)



In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

## **InceptionV3**

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import SGD



def train_model_with_generator(train_gen, batch_size=32, epochs=15):
    # Create a new model
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False

    last_layer = base_model.get_layer('mixed10')
    last_output = last_layer.output
    x = GlobalAveragePooling2D()(last_output)
    x = Dense(1024, activation='relu')(x)
    x = Dense(120, activation='softmax')(x)
    
    # Create the full model
    model = Model(inputs=base_model.input, outputs=x)

    # Compile the model
    sgd = SGD(
    learning_rate=0.001,  # Consider increasing this to 0.001 or 0.01 if training is slow
    momentum=0.9,
    nesterov=False,
    name='SGD'  # Correctly specify the name as a string
    )

# Compile the model with the SGD optimizer
    model.compile(
    optimizer=sgd,
    loss='categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' if your labels are integers
    metrics=['accuracy']
)
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    
    
    steps_per_epoch = len(X_train) // batch_size  # Number of batches per epoch


    # Train the model
    history = model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_val , y_val),
    epochs=epochs,
    callbacks=[early_stopping,lr_callback],
        )

    
    return history,model
history1,model = train_model_with_generator(train_generator)
plot_acc(history1,model)
plot_loss(history1,model)
# history2 = train_model_with_generator(train_generator2)
# plot_acc(history2,model)
# plot_loss(history2,model)
# history3 = train_model_with_generator(train_generator3)
# plot_acc(history3,model)
# plot_loss(history3,model)



In [ ]:
model.save('/kaggle/working/InceptionV3.h5')


# Evaluation 

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/kaggle/working/InceptionV3.h5')

In [ ]:
model.summary()

In [ ]:
IMAGE_SIZE = (224, 224)

def load_data(image_dir_test):
    """
    Load and preprocess the data:
    - Train images and their labels.
    - Test images for evaluation.
    """

    def process_images(image_dir):
        images = []
        print("Loading images from {}".format(image_dir))
        for image_file in tqdm(os.listdir(image_dir)):
            img_path = os.path.join(image_dir, image_file)

            # Open and resize the image
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, IMAGE_SIZE)

            images.append(image)

        return np.array(images, dtype='float32')

    # Process train images and labels
    test_images = process_images(image_dir_test)
    ids = []
    for image_file in tqdm(os.listdir(image_dir_test)):
        image_id = image_file.split('.')[0]
        ids.append(image_id)
    return test_images, ids


image_dir_test  = '/kaggle/input/dog-breed-identification/test'
test_images, ids = load_data(image_dir_test)

In [ ]:
print(f"Test images shape: {test_images.shape} ")

In [ ]:
len(ids)

In [ ]:
test_images[0]

In [ ]:
test_images = test_images / 255.0

In [ ]:
test_images[0]

In [ ]:
predictions = model.predict(test_images)

In [ ]:
len(predictions)

In [ ]:
predicted_breed = []
for i in range(len(predictions)) :
    predicted_breed.append(class_names_label[np.argmax(predictions[i])])
predicted_breed[:5]

In [ ]:
breed_names = class_names_label.values() 
breed_names


# Display Some Predictions

In [ ]:
def display_predictions(test_images,predicted_breed, num_images=5):
    """
    Displays a specified number of test images with their predicted breed labels.

    Parameters:
    - test_images (array-like): Rescaled test images, shape (num_samples, height, width, channels).
    - predictions (array-like): Predicted class indices for the test images.
    - predicted_breed (list): List mapping class indices to breed names.
    - num_images (int): Number of images to display.
    """
    plt.figure(figsize=(15, 5))  

    for i in range(num_images):
        plt.subplot(1, num_images, i + 1) 
        image = test_images[i]
        plt.imshow(image)
        plt.axis('off') 
        breed_name = predicted_breed[i]
        plt.title(breed_name, fontsize=12)
    
    plt.tight_layout()
    plt.show()
display_predictions(test_images,  predicted_breed, num_images=5)

# Submission 

In [ ]:
def create_submission_csv(image_ids, predictions, breed_names, output_file='submission.csv'):
    """
    Creates a CSV file for submission with columns: id, breed1, breed2, ..., breedN.

    Parameters:
    - image_ids (list): List of image IDs.
    - predictions (array-like): 2D array of predicted probabilities, shape (num_samples, num_breeds).
    - breed_names (list): List of breed names corresponding to the columns.
    - output_file (str): Name of the output CSV file.
    """
    # Create a DataFrame with image_ids as the 'id' column
    df = pd.DataFrame(predictions, columns=breed_names)
    df.insert(0, 'id', image_ids)  # Insert the 'id' column at the beginning
    df.to_csv(output_file, index=False)
    print(f"CSV file saved as {output_file}")
create_submission_csv(ids, predictions, breed_names, output_file='submission.csv')